In [1]:
%reload_ext autoreload
%autoreload 2

Failed to read module file 'C:\Python\Python311\Lib\pydoc_data\topics.py' for module 'pydoc_data.topics': UnicodeDecodeError
Traceback (most recent call last):
  File "c:\Users\Braian\OneDrive\Escritorio\GitHub\CEIA\feinn_project\.venv\Lib\site-packages\IPython\core\extensions.py", line 62, in load_extension
    return self._load_extension(module_str)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Braian\OneDrive\Escritorio\GitHub\CEIA\feinn_project\.venv\Lib\site-packages\IPython\core\extensions.py", line 77, in _load_extension
    mod = import_module(module_str)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Python\Python311\Lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1206, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1178, in _find_and_load
  File "<frozen importlib._bootst

In this notebook, we generate finite element data for training neural operators.

### General Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from pathlib import Path
from mesher import Mesh2D
from conditions import BoundaryCondition
from materials import LinearElastic
from solver import NFEA

In [4]:
def get_periodic_pairs(mesh: 'Mesh2D', tol: float = 1e-8):
    """
    Generate periodic node pairs for a 2D mesh based on its geometry and node groups.
    """

    coords = mesh.coordinates  # shape (nnod, 2)

    # Get boundary nodes
    x_min, x_max = coords[:, 0].min(), coords[:, 0].max()
    y_min, y_max = coords[:, 1].min(), coords[:, 1].max()

    left_nodes   = mesh.find_nodes(x_min, axis=1, tol=tol)
    right_nodes  = mesh.find_nodes(x_max, axis=1, tol=tol)
    bottom_nodes = mesh.find_nodes(y_min, axis=2, tol=tol)
    top_nodes    = mesh.find_nodes(y_max, axis=2, tol=tol)

    # LEFT & RIGHT pairs (same Y-value)
    y_to_right = {round(coords[nid-1, 1], 8): nid for nid in right_nodes}

    left_right_pairs = []
    for nid_l in left_nodes:
        y_key = round(coords[nid_l-1, 1], 8)
        if y_key in y_to_right:
            left_right_pairs.append((nid_l, y_to_right[y_key]))

    # BOTTOM & TOP pairs (same X-value)
    x_to_top = {round(coords[nid-1, 0], 8): nid for nid in top_nodes}

    bottom_top_pairs = []
    for nid_b in bottom_nodes:
        x_key = round(coords[nid_b-1, 0], 8)
        if x_key in x_to_top:
            bottom_top_pairs.append((nid_b, x_to_top[x_key]))

    # Sort by coordinates
    left_right_pairs.sort(key=lambda p: coords[p[0]-1, 1])   # sort by Y
    bottom_top_pairs.sort(key=lambda p: coords[p[0]-1, 0])   # sort by X

    return left_right_pairs, bottom_top_pairs

In [8]:
from conditions import PeriodicBoundaryCondition

soft = LinearElastic(emod =  20*1e3, nu = 0.25)
hard = LinearElastic(emod = 200*1e3, nu = 0.25)

delta_x = 0.10      # RVE size in X direction in (mm)
delta_y = 0.10      # RVE size in Y direction in (mm)

meshes_folder = Path("./meshes")
strain_folder = Path("./strain_histories")

mesh_files  = sorted(meshes_folder.glob("*.med"))               # mesh 1, mesh 2, ...
strain_files = sorted(strain_folder.glob("strain_*.csv"))       # strain_1.csv, strain_2.csv, ...

# batch to solve
b_ini = 1       # inclusive
b_end = 1       # inclusive

for idx in range(b_ini, b_end + 1):
    if idx < b_ini:
        continue
    if idx > b_end:
        break
    
    mesh_file = mesh_files[idx-1]
    strain_file = strain_files[idx-1]

    print(f"Processing {idx}/{b_end} → {mesh_file.name}")
    
    # Read mesh
    mesh = Mesh2D.from_salome_med(filepath=mesh_file, verbose=False)
    left_right_pairs, bottom_top_pairs = get_periodic_pairs(mesh)

    # Define boundary conditions
    bcs = {
        'origin_n': [BoundaryCondition(dof=1, value=0.0),
                    BoundaryCondition(dof=2, value=0.0)],  
        }
    
    # Define periodic boundary conditions
    strain_data = np.genfromtxt(
            strain_file,
            delimiter=',',
            names=True,
            dtype=float,
            skip_header=0
        )

    exx = strain_data['exx']
    eyy = strain_data['eyy']
    gxy = strain_data['gxy']
    n_steps = len(exx)

    pbc = PeriodicBoundaryCondition(
        exx=exx[0],
        eyy=eyy[0],
        gxy=gxy[0],
        left_right_pairs=left_right_pairs,
        bottom_top_pairs=bottom_top_pairs,
        delta_x=delta_x,
        delta_y=delta_y
        )

    mpcs = pbc.get_constraints()

    # Define material field mapping
    matfield = {'soft_s': soft, 
                'hard_s': hard}

    # Instance FEA solver
    fem_solver = NFEA(mesh = mesh, 
                        bcs = bcs,
                        matfld = matfield,
                        verbose = False
                        )
    break 
    # Solve for each step
    for step in range(n_steps):
        pbc.update_strains(new_exx=exx[step], new_eyy=eyy[step], new_gxy=gxy[step])
        mpcs = pbc.get_constraints()
        fem_solver.run_complete(nsteps=1)


Processing 1/1 → RVE_8x8_f0.40_case01.med
